In [4]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df = pd.DataFrame({'a': [0,1,2], 'b': [3,4,5]})
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))

NameError: ignored

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install transformers
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import tensorflow as tf
import tensorflow.keras

In [ ]:
test_split = 0.2
batch_size = 64
epochs = 2
learning_rate = 1e-4

In [ ]:
#sorted function
dataset_path = '/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary'
articles_path = sorted(glob.glob(os.path.join(dataset_path, "News Articles/*/*.txt")))
summaries_path = sorted(glob.glob(os.path.join(dataset_path, "Summaries/*/*.txt")))
print(articles_path[3], summaries_path[3])


IndexError: ignored

In [ ]:
articles = []
summaries = []

for article_path, summary_path in zip(articles_path, summaries_path):
    with open(article_path, "r", encoding="ISO-8859-1") as article_file:
        article_text = article_file.read()
        articles.append(article_text)

    with open(summary_path, "r", encoding="ISO-8859-1") as summary_file:
        summary_text = summary_file.read()
        summaries.append(summary_text)

In [ ]:
categories_list = os.listdir(os.path.join(dataset_path, 'News Articles'))
categories = []
def print_file_data():
    for category in categories_list:
        article_paths = glob.glob(os.path.join(dataset_path, "News Articles", category, "*.txt"))
        summary_paths = glob.glob(os.path.join(dataset_path, "Summaries", category, "*.txt"))

        for i in article_paths:
            categories.append(category)

        if len(article_paths) != len(summary_paths):
            print("Length of dataset not equal for subdirectory :", category)
        else:
            print(f"Found {len(article_paths)} articles and {len(summary_paths)} summaries in the subdirectory : {category}")

print_file_data()

In [ ]:
articles[2]

In [ ]:
summaries[2]

In [ ]:
df = pd.DataFrame({
    'Articles' : articles,
    'Summaries' : summaries,
    #'Categories' : categories
})

df.head()

In [ ]:
#category = df.groupby("Categories").size()
#print(category)
#plt.figure(figsize=(10,6))
#sns.barplot(x=category.index, y=category)
#plt.title("Category wise distribution of Articles")
#plt.show()

In [ ]:
df['Article Length'] = df["Articles"].apply(lambda x: len(x.split()))
df['Summary Length'] = df["Summaries"].apply(lambda x: len(x.split()))

In [ ]:
df

In [ ]:
import transformers
from transformers import TFBartForConditionalGeneration, BartTokenizer

In [ ]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = TFBartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [ ]:
tokenizer

In [ ]:
model

In [ ]:
filters = '[!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n]'
df["Articles"] = df["Articles"].str.replace('.', ' ', regex=True)
df['Articles'] = df["Articles"].str.replace(filters, '', regex=True)
df["Summaries"] = df["Summaries"].str.replace('.', ' ', regex=True)
df['Summaries'] = df["Summaries"].str.replace(filters, '', regex=True)

In [ ]:
df["Articles"][3]

In [ ]:
from sklearn.model_selection import train_test_split
X = df["Articles"]
Y = df["Summaries"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train, Y_train], axis=1)
test_df = pd.concat([X_test, Y_test], axis=1)

In [ ]:
train_df

In [ ]:
print((X_train.shape, Y_train.shape), (X_test.shape, Y_test.shape))

In [ ]:
from transformers import InputExample, InputFeatures
def convert_data_to_examples(train, test, data, label):
    train_input_examples = train.apply(lambda x: InputExample(
        guid = None,
        text_a = x[data],
        text_b = None,
        label = x[label]
    ), axis=1)

    test_input_examples = test.apply(lambda x: InputExample(
        guid = None,
        text_a = x[data],
        text_b = None,
        label = x[label]
    ), axis=1)

    return train_input_examples, test_input_examples

In [ ]:
train_input_examples, test_input_examples = convert_data_to_examples(train_df, test_df, "Articles", "Summaries")

In [ ]:
train_input_examples

In [ ]:
test_input_examples

In [ ]:
train_input_examples[6]

In [ ]:
#'Run this cell to see how to tokenizer.encode_plus works'


# input_dict = tokenizer.encode_plus(
#             list(train_input_examples[6].text_a),
#             add_special_tokens = True, # adds <start> and <end> tokens
#             max_length = 128, # defining max_len for padding
#             return_token_type_ids = True, # return token_type_ids
#             pad_to_max_length = True, # pads the sentences shorter than max_len
#             return_attention_mask = True, # return attention mask (tells model if the padded token is actual sentence or just added for dimension coorection of instance)
#             truncation = True
#         )
# input_dict

In [ ]:
# Tokenizing examples and converting into tensorflow dataset
def convert_examples_to_tf_dataset(examples, tokenizer, max_len=1024):
    features = [] #     Array to store the features

    for example in examples:
        input_dict = tokenizer.encode_plus(
            list(example.text_a),
            add_special_tokens = True, # adds <start> and <end> tokens
            max_length = max_len, # defining max_len for padding
            return_token_type_ids = True, # return token_type_ids
            pad_to_max_length = True, # pads the sentences shorter than max_len
            return_attention_mask = True, # return attention mask (tells model if the padded token is actual sentence or just added for dimension coorection of instance)
            truncation = True
        )
#       Returning all the params
        input_ids, attention_masks = (input_dict["input_ids"], input_dict["attention_mask"])



#         Same processing for labels
        label_dict = tokenizer.encode_plus(
            list(example.label),
            add_special_tokens = True, # adds <start> and <end> tokens
            max_length = max_len, # defining max_len for padding
            return_token_type_ids = True, # return token_type_ids
            pad_to_max_length = True, # pads the sentences shorter than max_len
            return_attention_mask = True, # return attention mask (tells model if the padded token is actual sentence or just added for dimension coorection of instance)
            truncation = True
        )

        label_ids = label_dict["input_ids"]

#       Adding these params in the features array to feed into the model later
        features.append(
            InputFeatures(
                input_ids = input_ids, attention_mask = attention_masks, label=label_ids
            )
        )


        def gen():
            for f in features:
                yield(
                    {
                        'input_ids':f.input_ids,
                        'attention_mask':f.attention_mask,
                    },
                    f.label[0],
                )



#     Returning data as tensroflow dataset for easy input in our model
    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int32),
        (
            {
                "input_ids":tf.TensorShape([None]),
                "attention_mask":tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


In [ ]:
# Preparing the dataset
train_data = convert_examples_to_tf_dataset(list(train_input_examples), tokenizer)
test_data = convert_examples_to_tf_dataset(list(train_input_examples), tokenizer)

In [ ]:
train_data.take(0)

In [ ]:
test_data

In [ ]:
# Preparing shuffles batches
train_data = train_data.shuffle(100).batch(32)
test_data = test_data.batch(32)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
def dataset_count(dataset):
    count = 0
    for _ in dataset:
        count+=1

    return count

print(dataset_count(train_data))
print(dataset_count(test_data))

In [ ]:
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
model.compile(Adam(learning_rate=learning_rate),loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=[SparseCategoricalAccuracy('accuracy')])

In [ ]:
model.fit(train_data, epochs=epochs, validation_data=test_data)